In [43]:
import json
import pandas as pd

filename = "../data/CIK0001649339.json"

with open(filename) as f:
    list_filings_json = json.load(f)
    
filings = pd.DataFrame()
filings["form"] = list_filings_json['filings']['recent']['form']
filings["accessionNumber"] = list_filings_json['filings']['recent']['accessionNumber']
filings["accessionNumber"] = filings["accessionNumber"].str.replace('-', '')
filings["filingDate"] = list_filings_json['filings']['recent']['filingDate']
filings["reportDate"] = list_filings_json['filings']['recent']['reportDate']
filings = filings[filings["form"]=="13F-HR"]
filings

In [44]:
filename = "../data/CIK0001649339.json"

In [45]:
!pwd

with open(filename) as f:
    list_filings_json = json.load(f)

/d1/serke/projects/python_tutorial/Numpy_tutorial


In [48]:
filings = pd.DataFrame()
filings["form"] = list_filings_json['filings']['recent']['form']
filings["accessionNumber"] = list_filings_json['filings']['recent']['accessionNumber']
filings["accessionNumber"] = filings["accessionNumber"].str.replace('-', '')
filings["filingDate"] = list_filings_json['filings']['recent']['filingDate']
filings["reportDate"] = list_filings_json['filings']['recent']['reportDate']
filings = filings[filings["form"]=="13F-HR"]
filings

,form,accessionNumber,filingDate,reportDate
0,13F-HR,000156761921003819,2021-02-16,2020-12-31
1,13F-HR,000156761920019679,2020-11-16,2020-09-30
2,13F-HR,000156761920015233,2020-08-14,2020-06-30
3,13F-HR,000156761920010271,2020-05-15,2020-03-31
9,13F-HR,000156761920003530,2020-02-14,2019-12-31
11,13F-HR,000156761919021303,2019-11-14,2019-09-30
13,13F-HR,000156761919016689,2019-08-14,2019-06-30
14,13F-HR,000156761919010955,2019-05-15,2019-03-31
15,13F-HR,000156761919004198,2019-02-14,2018-12-31
16,13F-HR,000114036116086026,2016-11-14,2016-09-30


In [75]:
import requests 
import xml.etree.ElementTree as et 
import yahooquery as yq
import time

def parse_XML(xml_file): 
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    df = pd.DataFrame()
    
    i=0
    for node in xroot: 
        for child in node:
        
            if (len(child)==0):
                df.loc[i,child.tag.split('}')[1]] = child.text
            else:
                prefix = child.tag.split('}')[1] + " - "
                
                for grandchild in child:
                    df.loc[i,prefix+grandchild.tag.split('}')[1]] = grandchild.text
        i = i+1
    return df

def get_ticker(row):
    
    time.sleep(1)
    
    try:
        data = yq.search(row["cusip"], first_quote=True)
        return data["symbol"]
    except:
        try:
            data = yq.search(row["nameOfIssuer"], first_quote=False)
            return data["quotes"][0]["symbol"]
        except:
            return None
    
def get_sector(row):
    
    time.sleep(1)
    if (row["ticker"]):
        try:
            info_ticker = yq.Ticker(row["ticker"]).all_modules
            return info_ticker[row["ticker"]]["assetProfile"]["sector"]
        except:
            return None
    else:
        return None
    

    
filings_detailed = pd.DataFrame()

for i,row in filings.iterrows():
    
    #get XML
    edgar_path    = "https://sec.gov/Archives/edgar/data/"
    company_numer = "1649339"
    url           = edgar_path + company_numer + "/" + row["accessionNumber"] + "/form13fInfoTable.xml"
    resp          = requests.get(url)
    
    with open(row["accessionNumber"]+'.xml', 'wb') as f: 
        f.write(resp.content) 
    
    #parse XML
    df_13f = parse_XML(row["accessionNumber"]+'.xml') 
    
    #get ticker, sector and compute portfolio importance
    df_13f["ticker"]      = df_13f.apply(get_ticker,axis=1)
    df_13f["sector"]      = df_13f.apply(get_sector,axis=1)
    df_13f["value"]       = pd.to_numeric(df_13f["value"])
    df_13f["% portfolio"] = df_13f["value"]/df_13f["value"].sum()
    
    #append detailed filing
    df_13f["reportDate"]  = row["reportDate"]
    
    if filings_detailed.empty:
        filings_detailed  = df_13f
    else:
        filings_detailed  = pd.concat([filings_detailed,df_13f])

In [54]:
def parse_XML(xml_file): 
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    df = pd.DataFrame()
    
    i=0
    for node in xroot: 
        for child in node:
        
            if (len(child)==0):
                df.loc[i,child.tag.split('}')[1]] = child.text
            else:
                prefix = child.tag.split('}')[1] + " - "
                
                for grandchild in child:
                    df.loc[i,prefix+grandchild.tag.split('}')[1]] = grandchild.text
        i = i+1
    return df

def get_ticker(row):
    
    time.sleep(1)
    
    try:
        data = yq.search(row["cusip"], first_quote=True)
        return data["symbol"]
    except:
        try:
            data = yq.search(row["nameOfIssuer"], first_quote=False)
            return data["quotes"][0]["symbol"]
        except:
            return None
    
def get_sector(row):
    
    time.sleep(1)
    if (row["ticker"]):
        try:
            info_ticker = yq.Ticker(row["ticker"]).all_modules
            return info_ticker[row["ticker"]]["assetProfile"]["sector"]
        except:
            return None
    else:
        return None
    

    
filings_detailed = pd.DataFrame()

for i,row in filings.iterrows():
    
    #get XML
    edgar_path="https://www.sec.gov/edgar/searchedgar/companysearch.html"
    url = edgar_path+row["accessionNumber"]+"/form13fInfoTable.xml"
    resp = requests.get(url) 
    
    with open(row["accessionNumber"]+'.xml', 'wb') as f: 
        f.write(resp.content) 
    
    #parse XML
    df_13f = parse_XML(row["accessionNumber"]+'.xml') 
    
    #get ticker, sector and compute portfolio importance
    df_13f["ticker"] = df_13f.apply(get_ticker,axis=1)
    df_13f["sector"] = df_13f.apply(get_sector,axis=1)
    df_13f["value"] = pd.to_numeric(df_13f["value"])
    df_13f["% portfolio"] = df_13f["value"]/df_13f["value"].sum()
    
    #append detailed filing
    df_13f["reportDate"] = row["reportDate"]
    
    if filings_detailed.empty:
        filings_detailed = df_13f
    else:
        filings_detailed = pd.concat([filings_detailed,df_13f])

ParseError: mismatched tag: line 37, column 2 (<string>)

In [87]:
import plotly.graph_objects as go

#get list of sectors 
sectors         = filings_detailed["sector"].unique()

#group by sector and filings
filings_sectors = filings_detailed.groupby(["reportDate","sector"]).sum()

#format sector allocation
x = filings_sectors.groupby(level=0).sum().index
x = [str(x) for x in x]
y = [[] for s in sectors]

for date, filing in filings_sectors.groupby(level=0):
    
    filing = filing.reset_index()
    
    for i in range(len(sectors)):
        try:
            y[i].append(100*filing.loc[filing["sector"] == sectors[i],"% portfolio"].values[0])
        except:
            y[i].append(0)
            
#holdings moves
filings_detailed            = filings_detailed.set_index("reportDate")
filings_detailed["shrsOrPrnAmt - sshPrnamt"] = pd.to_numeric(filings_detailed["shrsOrPrnAmt - sshPrnamt"])
filings_detailed["putCall"] = filings_detailed["putCall"].fillna("")
filings_detailed["holding"] = filings_detailed["nameOfIssuer"].astype(str) + " " + filings_detailed["putCall"].astype(str)

holdings               = filings_detailed["holding"].unique()
holdings_timeline      = pd.DataFrame(index=holdings,columns=x)
filings_detailed_moves = pd.DataFrame()

KeyError: "None of ['reportDate'] are in the columns"

In [90]:
filings_detailed

,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt - sshPrnamt,shrsOrPrnAmt - sshPrnamtType,investmentDiscretion,otherManager,votingAuthority - Sole,votingAuthority - Shared,votingAuthority - None,putCall,ticker,sector,% portfolio,holding
reportDate,,,,,,,,,,,,,,,,
2020-12-31,ALLSTATE CORP,COM,020002101,8245,75000,SH,DFND,"1,2",75000,0,0,,ALL,Financial Services,0.036694,ALLSTATE CORP
2020-12-31,ARES CAPITAL CORP,COM,04010L103,2111,125000,SH,DFND,"1,2",125000,0,0,,ARCC,Financial Services,0.009395,ARES CAPITAL CORP
2020-12-31,CITIGROUP INC,COM NEW,172967424,33272,539600,SH,DFND,"1,2",539600,0,0,Call,C,Financial Services,0.148074,CITIGROUP INC Call
2020-12-31,CORECIVIC INC,COM,21871N101,6550,1000000,SH,DFND,"1,2",1000000,0,0,,CXW,Real Estate,0.029150,CORECIVIC INC
2020-12-31,DESIGNER BRANDS INC,CL A,250565108,6885,900000,SH,DFND,"1,2",900000,0,0,,DBI,Consumer Cyclical,0.030641,DESIGNER BRANDS INC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31,INTERNATIONAL BUSINESS MACHS,COM,459200101,6881,50000,SH,SOLE,NaN,50000,0,0,,IBM,Technology,0.086078,INTERNATIONAL BUSINESS MACHS
2015-12-31,MENS WEARHOUSE INC,COM,587118100,5872,400000,SH,SOLE,NaN,400000,0,0,,None,None,0.073456,MENS WEARHOUSE INC
2015-12-31,NEXPOINT RESIDENTIAL TR INC,COM,65341D102,8781,670800,SH,SOLE,NaN,670800,0,0,,NXRT,Real Estate,0.109846,NEXPOINT RESIDENTIAL TR INC


In [92]:
date

'2020-12-31'

In [93]:
filing

,reportDate,sector,value,shrsOrPrnAmt - sshPrnamt,% portfolio
0,2020-12-31,Basic Materials,3376,776177,0.015025
1,2020-12-31,Communication Services,23821,1745000,0.106013
2,2020-12-31,Consumer Cyclical,20539,2100000,0.091407
3,2020-12-31,Consumer Defensive,31971,851115,0.142284
4,2020-12-31,Energy,17233,1750000,0.076694
5,2020-12-31,Financial Services,51173,989600,0.227741
6,2020-12-31,Healthcare,31079,844300,0.138315
7,2020-12-31,Real Estate,37197,3995152,0.165542
8,2020-12-31,Technology,8309,150000,0.036979
